In [24]:
import salabim as sim
#ty
# from Customer_class import Customer
class InitModel:
    def __init__(
        self,
        cart_capacity=45,
        basket_capacity=500,
        cart_distribution=0.8,
        bread_clerks_capacity=4,
        bread_clerks_time=120,  # in seconds
        cheese_diary_clerks_capacity=3,
        cheese_diary_clerks_time=60,  # in seconds
        number_of_checkouts=3,
        time_per_item_lambda=1.1,  # Exponential distribution parameter
        payment_time_range=[40, 60],  # Uniform distribution [min, max]
        fruit_and_vegetables_range=[4, 22, 10],  # Triangular [min, mode, max]
        meat_and_fish_range=[0, 9, 4],  # Triangular [min, mode, max]
        bread_range=[1, 10, 4],  # Triangular [min, mode, max]
        cheese_and_dairy_range=[1, 11, 3],  # Triangular [min, mode, max]
        canned_and_packed_food_range=[6, 35, 17],  # Triangular [min, mode, max]
        frozen_food_range=[2, 19, 8],  # Triangular [min, mode, max]
        drinks_range=[1, 20, 9],  # Triangular [min, mode, max]
        item_taking_time_range=[20, 30],  # Uniform distribution [min, max]
        route_distribution=0.8,
        customer_to_log = 500
    ):
        self.env = sim.Environment(trace=True)  # Placeholder for the simulation environment, to be set later

        # Shopping baskets and carts
        self.carts = sim.Resource('carts', capacity=cart_capacity)
        self.baskets = sim.Resource('baskets', capacity=basket_capacity)

        # Cart/Basket Distribution
        self.cart_basket_distribution = sim.Pdf((self.carts, cart_distribution, self.baskets, 1 - cart_distribution))

        # Clerks
        self.bread_clerks = sim.Resource('bread_clerks', capacity=bread_clerks_capacity)
        self.bread_time_distribution = sim.Exponential(bread_clerks_time)

        self.cheese_and_dairy_clerks = sim.Resource('cheese_and_dairy_clerks', capacity=cheese_diary_clerks_capacity)
        self.cheese_and_dairy_time_distribution = sim.Exponential(cheese_diary_clerks_time)

        # Checkouts
        self.number_of_checkouts = number_of_checkouts
        self.time_per_item_distribution = sim.Exponential(time_per_item_lambda)
        self.payment_time_distribution = sim.Uniform(payment_time_range[0], payment_time_range[1])

        # Distributions of items per customer
        self.fruit_and_vegetables_distribution = sim.Triangular(*fruit_and_vegetables_range)
        self.meat_and_fish_distribution = sim.Triangular(*meat_and_fish_range)
        self.bread_distribution = sim.Triangular(*bread_range)
        self.cheese_and_dairy_distribution = sim.Triangular(*cheese_and_dairy_range)
        self.canned_and_packed_food_distribution = sim.Triangular(*canned_and_packed_food_range)
        self.frozen_food_distribution = sim.Triangular(*frozen_food_range)
        self.drinks_distribution = sim.Triangular(*drinks_range)

        # Time to take item off the shelf
        self.item_taking_distribution = sim.Uniform(*item_taking_time_range)

        # Route choice distribution
        self.route1 = [
            "fruit_and_vegetables",
            "meat_and_fish",
            "bread",
            "cheese_and_dairy",
            "canned_and_packed_food",
            "frozen_foods",
            "drinks"
        ]  # ABCDEF, 80%

        self.route2 = [
            "meat_and_fish",
            "bread",
            "cheese_and_dairy",
            "fruit_and_vegetables",
            "canned_and_packed_food",
            "frozen_foods",
            "drinks"
        ]  # BCDEFG, 20%

        self.route_distribution_pdf = sim.Pdf((self.route1, route_distribution, self.route2, 1 - route_distribution))
        
        self.customer_to_log = customer_to_log
        
                #Checkout creation
        self.checkouts = []
        for i in range(self.number_of_checkouts):
            self.checkouts.append(self.env.Resource(f"checkout_clerk{i}", capacity = 1)) #3, 1.1s per item avg. payment 40-60s
        
        class Customer(sim.Component):
            """
            Customer class for the supermarket.
            Traverses the store to via its route to fulfill its shopping_list, while carrying either a shopping basket or cart.
            """
            def __init__(self, model = self,*args, **kwargs):
                super().__init__(*args, **kwargs)
                self.route = model.route_distribution_pdf.sample()
                self.shopping_list = {
                    "fruit_and_vegetables": int(round(model.fruit_and_vegetables_distribution.sample())),
                    "meat_and_fish": int(round(model.meat_and_fish_distribution.sample())),
                    "bread": int(round(model.bread_distribution.sample())),
                    "cheese_and_dairy": int(round(model.cheese_and_dairy_distribution.sample())),
                    "canned_and_packed_food": int(round(model.canned_and_packed_food_distribution.sample())),
                    "frozen_foods": int(round(model.frozen_food_distribution.sample())),
                    "drinks": int(round(model.drinks_distribution.sample())),
                }
                self.carrying = None
                self.actions_log = []
                self.model = model
        
        
            def log_action(self, action):
                """Helper function to log an action with the current time."""
                self.actions_log.append((model.env.now(), action))
        
            def process(self):
                """"
                Process determines what the customer will do. At the start they will take a cart or basket. Afterwards they will traverse their route and take the items they need according to their shopping list. If they have finished their route (when progress is equal to the length of the shopping list), they will go to the checkout.
                """
                self.start_shopping()
                for next_product in self.route:
                    if self.shopping_list[next_product] > 0:
                        self.get_product(next_product)
                self.go_to_checkout()
        
            def start_shopping(self):
                """ Get either a shopping cart or basket"""
                want_to_carry = model.cart_basket_distribution.sample()
                self.log_action(f"Entered cart/basket queue for {want_to_carry}")
                self.request(want_to_carry)
                self.log_action(f"Got {want_to_carry}")
                self.carrying = want_to_carry
        
            def go_to_checkout(self):
                """Proceed to the emptiest queue in the checkout and wait while items are processed. Returns shopping cart/basket afterwards."""
                # enter emptiest queue
                emptiest_queue = min(self.model.checkouts, key=lambda checkout: checkout.requesters().length())
                self.log_action(f"Entered checkout queue {emptiest_queue}")
                self.request(emptiest_queue)
                self.log_action(f"Started checking out")
                item_scan_time = sum(model.time_per_item_distribution.sample() for _ in range(sum(self.shopping_list.values())))
                self.hold(
                    item_scan_time + model.payment_time_distribution.sample())  # hold the customer for scanning all items and during payment
                self.log_action(f"Finished checking out")
                # return cart/basket (implicit since process finishes)
        
                # print log if we want to debug
        
                if model.customer_to_log:
                    if self.name() == f"customer.{model.customer_to_log}":
                        print(self.carrying.claimers().print_info())
                        print(f"Customer's Action Log for customer {self.name()}:")
                        for time, action in self.actions_log:
                            print(f"At time {time}, customer: {action}")
        
            def get_product(self, product):
                """
                Function to get the required number of {product}.
                Customer holds while the products are taken.
                Special cases for cheese_and_dairy and bread as they require clerks.
                """
                if product == "cheese_and_dairy":
                    self.log_action(f"requesting cheese and dairy")
                    self.request(model.cheese_and_dairy_clerks)
                    self.log_action(f"Being helped for cheese and dairy")
                    self.hold(model.cheese_and_dairy_time_distribution.sample())
                    self.log_action(f"Got cheese and dairy")
                    self.release(model.cheese_and_dairy_clerks)
                elif product == "bread":
                    self.log_action(f"requesting bread")
                    self.request(model.bread_clerks)
                    self.log_action(f"Being helped for bread")
                    self.hold(model.bread_time_distribution.sample())
                    self.log_action(f"Got bread")
                    self.release(model.bread_clerks)
                else:
                    amount = self.shopping_list[product]
                    self.log_action(f"Getting {product}")
                    for _ in range(amount):
                        self.hold(model.time_per_item_distribution.sample())
                        self.log_action(f"Got {product}")
        self.cus =Customer
#customer generation
    def run_model(self):
        customer_distribution = [30, 80, 110, 90, 80, 70, 80, 90, 100, 120, 90, 40] #Expected total = 980
        for index, customer_count  in enumerate(customer_distribution):
            self.env.ComponentGenerator(self.cus, iat=self.env.Exponential(3600/customer_count,env=self), at=index*60*60, duration=60*60) #assumes time in seconds
        

        
        #Run for the full day (duration is in seconds)
        self.env.run(duration=10*len(customer_distribution))
        return self.bread_clerks.print_statistics()

#Output statistics:
# for checkout in checkouts:
#     checkout.length.print_histograms()
#carts.print_statistics()
# carts.print_info()
# bread_clerks.print_statistics()
# bread_clerks.print_info()
# cheese_and_dairy_clerks.print_statistics()
# cheese_and_dairy_clerks.print_info()
#customer_basketcart_distribution_monitor.print_histogram(values=True)


In [25]:
model = InitModel()

line#        time current component    action                               information
------ ---------- -------------------- -----------------------------------  ------------------------------------------------
                                       line numbers refers to               1546863120.py
   28                                  default environment initialize       
   28                                  main create                          
   28       0.000 main                 current                              
   31                                  carts create                         capacity=45
   32                                  baskets create                       capacity=500
   38                                  bread_clerks create                  capacity=4
   41                                  cheese_and_dairy_clerks create       capacity=3
   89                                  checkout_clerk0 create               capacity=1
   89                       

In [26]:
model.run_model()

  188                                  Customer.generator.0 create          
  188                                  Customer.generator.0 activate +396.086 scheduled for 396.086  process=do_iat_yieldless
  188                                  Customer.generator.1 create          
  188                                  Customer.generator.1 activate +3612.098 scheduled for 3612.098  process=do_iat_yieldless
  188                                  Customer.generator.2 create          
  188                                  Customer.generator.2 activate +7228.507 scheduled for 7228.507  process=do_iat_yieldless
  188                                  Customer.generator.3 create          
  188                                  Customer.generator.3 activate +10810.285 scheduled for 10810.285  process=do_iat_yieldless
  188                                  Customer.generator.4 create          
  188                                  Customer.generator.4 activate +14552.952 scheduled for 14552.952

In [27]:
def EMA_func(
        cart_capacity=45,
        basket_capacity=500,
        cart_distribution=0.8,
        bread_clerks_capacity=4,
        bread_clerks_time=120,  # in seconds
        cheese_diary_clerks_capacity=3,
        cheese_diary_clerks_time=60,  # in seconds
        number_of_checkouts=3,
        time_per_item_lambda=1.1,  # Exponential distribution parameter
        payment_time_range=[40, 60],  # Uniform distribution [min, max]
        fruit_and_vegetables_range=[4, 22, 10],  # Triangular [min, mode, max]
        meat_and_fish_range=[0, 9, 4],  # Triangular [min, mode, max]
        bread_range=[1, 10, 4],  # Triangular [min, mode, max]
        cheese_and_dairy_range=[1, 11, 3],  # Triangular [min, mode, max]
        canned_and_packed_food_range=[6, 35, 17],  # Triangular [min, mode, max]
        frozen_food_range=[2, 19, 8],  # Triangular [min, mode, max]
        drinks_range=[1, 20, 9],  # Triangular [min, mode, max]
        item_taking_time_range=[20, 30],  # Uniform distribution [min, max]
        route_distribution=0.8,
        customer_to_log = 500
):
    [para.default for para in list(dict(inspect.signature(EMA_func).parameters).values())]
    pass

[45,
 500,
 0.8,
 4,
 120,
 3,
 60,
 3,
 1.1,
 [40, 60],
 [4, 22, 10],
 [0, 9, 4],
 [1, 10, 4],
 [1, 11, 3],
 [6, 35, 17],
 [2, 19, 8],
 [1, 20, 9],
 [20, 30],
 0.8,
 500]

In [54]:
import inspect
list(dict(inspect.signature(EMA_func).parameters).values())[0].default

45

In [ ]:
[para.default for para in list(dict(inspect.signature(EMA_func).parameters).values())]

In [53]:
help(list(dict(inspect.signature(EMA_func).parameters).values())[0])

Help on Parameter in module inspect object:

class Parameter(builtins.object)
 |  Parameter(name, kind, *, default, annotation)
 |  
 |  Represents a parameter in a function signature.
 |  
 |  Has the following public attributes:
 |  
 |  * name : str
 |      The name of the parameter as a string.
 |  * default : object
 |      The default value for the parameter if specified.  If the
 |      parameter has no default value, this attribute is set to
 |      `Parameter.empty`.
 |  * annotation
 |      The annotation for the parameter if specified.  If the
 |      parameter has no annotation, this attribute is set to
 |      `Parameter.empty`.
 |  * kind : str
 |      Describes how argument values are bound to the parameter.
 |      Possible values: `Parameter.POSITIONAL_ONLY`,
 |      `Parameter.POSITIONAL_OR_KEYWORD`, `Parameter.VAR_POSITIONAL`,
 |      `Parameter.KEYWORD_ONLY`, `Parameter.VAR_KEYWORD`.
 |  
 |  Methods defined here:
 |  
 |  __eq__(self, other)
 |      Return self==va

In [41]:
help(inspect.signature(EMA_func).parameters)

Help on mappingproxy object:

class mappingproxy(object)
 |  Methods defined here:
 |  
 |  __contains__(self, key, /)
 |      Return key in self.
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __getitem__(self, key, /)
 |      Return self[key].
 |  
 |  __gt__(self, value, /)
 |      Return self>value.
 |  
 |  __ior__(self, value, /)
 |      Return self|=value.
 |  
 |  __iter__(self, /)
 |      Implement iter(self).
 |  
 |  __le__(self, value, /)
 |      Return self<=value.
 |  
 |  __len__(self, /)
 |      Return len(self).
 |  
 |  __lt__(self, value, /)
 |      Return self<value.
 |  
 |  __ne__(self, value, /)
 |      Return self!=value.
 |  
 |  __or__(self, value, /)
 |      Return self|value.
 |  
 |  __repr__(self, /)
 |      Return repr(self).
 |  
 |  __reversed__(...)
 |      D.__reversed__() -> reverse iterator
 |